# Gray-Level Co-occurrence Matrix，GLCM 灰度共生矩阵

In [1]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch

In [2]:
def fast_glcm(img, vmin=0, vmax=255, nbit=8, kernel_size=5):
    """
    img: 传入的图像
    vmin: 灰度的最小值
    vmax: 灰度的最大值
    nbit： 间隔
    kernel_size: 卷积核的大小
    """
    mi, ma = vmin, vmax
    ks = kernel_size
    h,w = img.shape

    # digitize
    bins = np.linspace(mi, ma+1, nbit+1)
    gl1 = np.digitize(img, bins) - 1
    gl2 = np.append(gl1[:,1:], gl1[:,-1:], axis=1)

    # make glcm
    glcm = np.zeros((nbit, nbit, h, w), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            mask = ((gl1==i) & (gl2==j))
            glcm[i,j, mask] = 1

    kernel = np.ones((ks, ks), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            glcm[i,j] = cv2.filter2D(glcm[i,j], -1, kernel)

    glcm = glcm.astype(np.float32)
    return glcm

In [4]:
def fast_glcm_mean(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    calc glcm mean
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    mean = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            mean += glcm[i,j] * i / (nbit)**2
    return mean

def fast_glcm_std(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    calc glcm std
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    mean = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            mean += glcm[i,j] * i / (nbit)**2

    std2 = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            std2 += (glcm[i,j] * i - mean)**2

    std = np.sqrt(std2)
    return std

In [5]:
def fast_glcm_contrast(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    计算对比度
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    cont = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            cont += glcm[i,j] * (i-j)**2
    return cont

In [6]:
def fast_glcm_dissimilarity(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    计算不相似度
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    diss = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            diss += glcm[i,j] * np.abs(i-j)
    return diss

In [7]:
def fast_glcm_homogeneity(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    计算均匀性
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    homo = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            homo += glcm[i,j] / (1.+(i-j)**2)
    return homo

In [8]:
def fast_glcm_ASM(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    ASM（Angular Second Moment）和能量（Energy）
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    asm = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            asm  += glcm[i,j]**2

    ene = np.sqrt(asm)
    return asm, ene

In [9]:
def fast_glcm_max(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    计算GLCM的最大值（Maximum）
    '''
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    max_  = np.max(glcm, axis=(0,1))
    return max_

In [10]:
def fast_glcm_entropy(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    计算GLCM的熵（Entropy）
    '''
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    pnorm = glcm / np.sum(glcm, axis=(0,1)) + 1./ks**2
    ent  = np.sum(-pnorm * np.log(pnorm), axis=(0,1))
    return ent

In [11]:
def mnist_on_glcm():
    nbit = 8
    ks = 5
    mi, ma = 0, 255

    ori_f = '../data/MNIST/processed/training.pt' #该数据集使用torchvision下载
    imgs, labels = torch.load(ori_f)

    glcm_mnist = []
    for img in imgs:
        img = fast_glcm_mean(img, vmin=mi, vmax=ma, nbit=nbit, ks=ks)
        glcm_mnist.append(img)
    glcm_mnist = torch.tensor(glcm_mnist)

    file = (glcm_mnist, labels)
    tar_f = '../data/mnist_custom/processed/mnist_glcm.pt'
    with open(tar_f, 'wb') as f:
        torch.save(file, f)
    print('shape ', glcm_mnist.shape)
    print('glcm on mnist generate success.')
    
def fashionmnist_on_glcm():
    nbit = 8
    ks = 5
    mi, ma = 0, 255

    ori_f = '../data/FashionMNIST/processed/training.pt'
    imgs, labels = torch.load(ori_f)

    glcm_mnist = []
    for img in imgs:
        img = fast_glcm_mean(img, vmin=mi, vmax=ma, nbit=nbit, ks=ks)
        glcm_mnist.append(img)
    glcm_mnist = torch.tensor(glcm_mnist)

    file = (glcm_mnist, labels)
    tar_f = '../data/FashionMNIST/processed/fashionmnist_glcm.pt'
    with open(tar_f, 'wb') as f:
        torch.save(file, f)
    print('shape ', glcm_mnist.shape)
    print('glcm on fashionmnist generate success.')

In [13]:
if __name__ == '__main__':
    import time
    t1 = time.time()
    mnist_on_glcm()
    fashionmnist_on_glcm()
    t2 = time.time()
    print('cost {}s'.format(t2 - t1))

shape  torch.Size([60000, 28, 28])
glcm on mnist generate success.
shape  torch.Size([60000, 28, 28])
glcm on fashionmnist generate success.
cost 102.83318877220154s
